In [5]:
import pandas as pd
from datetime import datetime
from datetime import timedelta
import numpy as np
import random
import json
import math
from typing import List, Tuple

In [248]:

def get_service_poll_date(file_path):
    all_info = file_path.split("_")
    service = all_info[2]
    poll_rate = all_info[3]
    date_concerned = datetime.strptime(all_info[4].split(".")[0], '%d-%m-%Y')

    print("Service is:",service)
    print("Poll rate is:",poll_rate)
    print("Date is:",date_concerned)
    print()

    return [service, poll_rate, date_concerned]

def filter_df_for_date(df, date_concerned, svc_start_time, svc_end_time):
    start_time = datetime.strptime(svc_start_time,'%H%M').time()
    end_time = datetime.strptime(svc_end_time,'%H%M').time()


    sched_start = datetime.combine(date_concerned, start_time)
    if end_time < start_time: #end time is past midnight
        sched_end = datetime.combine(date_concerned + timedelta(days=1), end_time)
    else:
        sched_end = datetime.combine(date_concerned, end_time)

    print("schedule starts at", sched_start)
    print("schedule ends at", sched_end)

    filtered_df = df[df["vehicleTimestamp"] > sched_start]
    filtered_df = df[df["vehicleTimestamp"] < sched_end]

    return filtered_df

def clean_data(orig_df):
    cleaned_df = orig_df.copy() 
    cleaned_df["vehicleTimestamp"] = pd.to_datetime(cleaned_df["vehicleTimestamp"], unit="s")
    cleaned_df.drop(["pippenId", "pippenCreatedAt", "pippenPollingRate", "tripStartTime", "tripStartDate", "tripRouteId"] , axis=1, inplace = True)
    cleaned_df["tripDirectionId"] = np.where((cleaned_df["vehicleLabel"] == "FX2 To Portland") | (cleaned_df["vehicleLabel"] == "FX2 To NW 5th & Hoyt") | (cleaned_df["vehicleLabel"] == "FX2 To NW Irving & 5th"), 1, 0)
    cleaned_df.dropna(subset=["vehicleStopID"], inplace=True)
    cleaned_df["vehicleStopID"] = cleaned_df["vehicleStopID"].astype(int)

    cleaned_df.drop_duplicates(inplace=True)
    cleaned_df.sort_values(by=['tripId', "vehicleStopSequence"], inplace=True)
    


        # if row["vehicleStopSequence"] >2:
        #     cleaned_df.drop(index, inplace = True)

    # for index, row in cleaned_df.groupby("tripId").head(1).sort_values("tripId").iterrows():
    #     print(row["vehicleStopSequence"])


    return cleaned_df


In [212]:
data = "gtfs_data_2_15s_07-10-2023.csv"
date_concerned = get_service_poll_date(data)[2]
df = pd.read_csv(data)
df = clean_data(df)
df = filter_df_for_date(df, date_concerned, "0500", "0100")
df = df[df["tripDirectionId"] ==0]
df.sort_values(["tripId","vehicleTimestamp"])


trip number: 49
trip id: 12782067
first stop sequence: 4

trip number: 86
trip id: 12782280
first stop sequence: 41

trip number: 87
trip id: 12782281
first stop sequence: 24

trip number: 88
trip id: 12782282
first stop sequence: 13

trip number: 183
trip id: 12782738
first stop sequence: 39

trip number: 184
trip id: 12782739
first stop sequence: 24

trip number: 185
trip id: 12782740
first stop sequence: 8

schedule starts at 2023-10-07 05:00:00
schedule ends at 2023-10-08 01:00:00


,tripId,tripDirectionId,vehicleTimestamp,vehicleLabel,vehicleID,vehicleLat,vehicleLong,vehicleBearing,vehicleStopSequence,vehicleStopID,vehicleCurrentStatus
2123,12782018,0,2023-10-07 12:32:34,FX2 To Gresham,4525,45.527695,-122.675570,178,1,9302,STOPPED_AT
2131,12782018,0,2023-10-07 12:33:04,FX2 To Gresham,4525,45.527695,-122.675570,178,1,9302,STOPPED_AT
2139,12782018,0,2023-10-07 12:33:31,FX2 To Gresham,4525,45.527550,-122.675560,178,2,9301,IN_TRANSIT_TO
2159,12782018,0,2023-10-07 12:34:37,FX2 To Gresham,4525,45.527550,-122.675560,178,2,9301,IN_TRANSIT_TO
2167,12782018,0,2023-10-07 12:35:10,FX2 To Gresham,4525,45.524770,-122.675460,178,2,9301,STOPPED_AT
...,...,...,...,...,...,...,...,...,...,...,...
56956,12782288,0,2023-10-07 09:34:30,FX2 To Cleveland Ave,4526,45.502920,-122.427734,89,41,8199,IN_TRANSIT_TO
56958,12782288,0,2023-10-07 09:34:57,FX2 To Cleveland Ave,4526,45.502956,-122.425460,88,42,14230,IN_TRANSIT_TO
56960,12782288,0,2023-10-07 09:35:27,FX2 To Cleveland Ave,4526,45.503010,-122.421710,88,42,14230,IN_TRANSIT_TO
1312,12782288,0,2023-10-07 09:35:57,FX2 To Cleveland Ave,4526,45.503036,-122.419170,88,42,14230,IN_TRANSIT_TO


In [165]:
df.groupby("tripId").head(1).sort_values("tripId")["vehicleStopSequence"].unique()

array([ 0,  1, 24, 13])

In [41]:
df[df["tripId"]==12782025]


,tripId,tripDirectionId,vehicleTimestamp,vehicleLabel,vehicleID,vehicleLat,vehicleLong,vehicleBearing,vehicleStopSequence,vehicleStopID,vehicleCurrentStatus
5333,12782025,0,2023-10-07 14:25:35,FX2 To Gresham,4524,45.527695,-122.675570,178,1,9302.0,STOPPED_AT
5355,12782025,0,2023-10-07 14:26:05,FX2 To Gresham,4524,45.527695,-122.675570,178,1,9302.0,STOPPED_AT
5377,12782025,0,2023-10-07 14:26:36,FX2 To Gresham,4524,45.527695,-122.675570,178,1,9302.0,STOPPED_AT
5399,12782025,0,2023-10-07 14:27:05,FX2 To Gresham,4524,45.527695,-122.675570,178,1,9302.0,STOPPED_AT
5421,12782025,0,2023-10-07 14:27:39,FX2 To Gresham,4524,45.527695,-122.675570,178,1,9302.0,STOPPED_AT
...,...,...,...,...,...,...,...,...,...,...,...
15006,12782025,0,2023-10-07 15:33:15,FX2 To Cleveland Ave,4524,45.502934,-122.426346,89,42,14230.0,IN_TRANSIT_TO
15032,12782025,0,2023-10-07 15:33:45,FX2 To Cleveland Ave,4524,45.502990,-122.422930,88,42,14230.0,IN_TRANSIT_TO
15058,12782025,0,2023-10-07 15:34:15,FX2 To Cleveland Ave,4524,45.503020,-122.420470,89,42,14230.0,IN_TRANSIT_TO
15084,12782025,0,2023-10-07 15:34:48,FX2 To Cleveland Ave,4524,45.503040,-122.418830,88,42,14230.0,STOPPED_AT


In [282]:
data2 = "gtfs_data_2_15s_12-10-2023.csv"
date_concerned = get_service_poll_date(data2)[2]
df2 = pd.read_csv(data2)
df2 = clean_data(df2)
df2 = filter_df_for_date(df2, date_concerned, "0500", "0100")
df2 = df2[df2["tripDirectionId"] ==0]
# df2.sort_values(["tripId","vehicleTimestamp"], inplace=True)
df2.sort_values(["tripId","vehicleStopSequence"], inplace=True)
df2.reset_index(inplace=True)

print("number of trips:",df2["tripId"].nunique())
print()
for index, row in df2.groupby("tripId").head(1).sort_values("vehicleTimestamp").reset_index().iterrows():
    if row["vehicleStopSequence"] == 1:
        print("trip number:",index)
        print("timestamp:", row["vehicleTimestamp"])
        print("trip id:",row["tripId"])
        print("first stop sequence:", row["vehicleStopSequence"])
        print()


schedule starts at 2023-10-12 05:00:00
schedule ends at 2023-10-13 01:00:00
number of trips: 72

trip number: 0
timestamp: 2023-10-12 06:59:38
trip id: 12845725
first stop sequence: 1

trip number: 3
timestamp: 2023-10-12 07:07:04
trip id: 12845726
first stop sequence: 1

trip number: 4
timestamp: 2023-10-12 07:28:54
trip id: 12845727
first stop sequence: 1

trip number: 5
timestamp: 2023-10-12 07:54:51
trip id: 12845728
first stop sequence: 1

trip number: 6
timestamp: 2023-10-12 08:09:52
trip id: 12845729
first stop sequence: 1

trip number: 7
timestamp: 2023-10-12 08:29:52
trip id: 12845730
first stop sequence: 1

trip number: 8
timestamp: 2023-10-12 12:01:05
trip id: 12845636
first stop sequence: 1

trip number: 9
timestamp: 2023-10-12 12:20:26
trip id: 12845637
first stop sequence: 1

trip number: 10
timestamp: 2023-10-12 12:44:43
trip id: 12845638
first stop sequence: 1

trip number: 11
timestamp: 2023-10-12 13:06:00
trip id: 12845639
first stop sequence: 1

trip number: 12
times

In [283]:
df2

,index,tripId,tripDirectionId,vehicleTimestamp,vehicleLabel,vehicleID,vehicleLat,vehicleLong,vehicleBearing,vehicleStopSequence,vehicleStopID,vehicleCurrentStatus
0,1191,12845636,0,2023-10-12 12:01:05,FX2 To Gresham,4517,45.527650,-122.675560,178,1,9302,STOPPED_AT
1,1199,12845636,0,2023-10-12 12:01:35,FX2 To Gresham,4517,45.527650,-122.675560,178,1,9302,STOPPED_AT
2,1207,12845636,0,2023-10-12 12:02:05,FX2 To Gresham,4517,45.527650,-122.675560,178,1,9302,STOPPED_AT
3,1215,12845636,0,2023-10-12 12:02:36,FX2 To Gresham,4517,45.527650,-122.675560,178,1,9302,STOPPED_AT
4,1223,12845636,0,2023-10-12 12:03:06,FX2 To Gresham,4517,45.527650,-122.675560,178,1,9302,STOPPED_AT
...,...,...,...,...,...,...,...,...,...,...,...,...
10333,10849,12845730,0,2023-10-12 09:29:43,FX2 To Cleveland Ave,4512,45.502922,-122.427500,89,41,8199,IN_TRANSIT_TO
10334,10851,12845730,0,2023-10-12 09:30:10,FX2 To Cleveland Ave,4512,45.502930,-122.426520,89,42,14230,IN_TRANSIT_TO
10335,10852,12845730,0,2023-10-12 09:30:29,FX2 To Cleveland Ave,4512,45.502945,-122.426150,85,42,14230,IN_TRANSIT_TO
10336,10854,12845730,0,2023-10-12 09:30:59,FX2 To Cleveland Ave,4512,45.503002,-122.422170,88,42,14230,IN_TRANSIT_TO


In [269]:
df2[df2["tripId"]== 12845725]



,tripId,tripDirectionId,vehicleTimestamp,vehicleLabel,vehicleID,vehicleLat,vehicleLong,vehicleBearing,vehicleStopSequence,vehicleStopID,vehicleCurrentStatus
7,12845725,0,2023-10-12 06:59:38,FX2 To Gresham,4529,45.527695,-122.675570,178,1,9302,STOPPED_AT
15,12845725,0,2023-10-12 07:00:08,FX2 To Gresham,4529,45.527695,-122.675570,178,1,9302,STOPPED_AT
31,12845725,0,2023-10-12 07:00:38,FX2 To Gresham,4529,45.527695,-122.675570,178,1,9302,STOPPED_AT
47,12845725,0,2023-10-12 07:01:08,FX2 To Gresham,4529,45.527500,-122.675560,178,2,9301,IN_TRANSIT_TO
55,12845725,0,2023-10-12 07:01:23,FX2 To Gresham,4529,45.527450,-122.675550,178,2,9301,IN_TRANSIT_TO
...,...,...,...,...,...,...,...,...,...,...,...
261,12845725,0,2023-10-12 07:57:54,FX2 To Cleveland Ave,4529,45.502926,-122.427000,89,41,8199,STOPPED_AT
25757,12845725,0,2023-10-12 07:53:54,FX2 To Cleveland Ave,4529,45.504414,-122.433370,90,41,8199,IN_TRANSIT_TO
273,12845725,0,2023-10-12 07:58:24,FX2 To Cleveland Ave,4529,45.502930,-122.426636,88,42,14230,IN_TRANSIT_TO
279,12845725,0,2023-10-12 07:58:33,FX2 To Cleveland Ave,4529,45.502937,-122.426280,85,42,14230,IN_TRANSIT_TO


In [112]:
df.groupby("tripId").head(1).sort_values("vehicleTimestamp")

,tripId,tripDirectionId,vehicleTimestamp,vehicleLabel,vehicleID,vehicleLat,vehicleLong,vehicleBearing,vehicleStopSequence,vehicleStopID,vehicleCurrentStatus
2,12782282,0,2023-10-07 06:59:47,FX2 To Gresham,4523,45.504803,-122.638750,90,13,1447.0,IN_TRANSIT_TO
3,12782281,0,2023-10-07 06:59:50,FX2 To Gresham,4524,45.504456,-122.566765,91,24,1359.0,IN_TRANSIT_TO
4,12782280,0,2023-10-07 07:00:08,FX2 To Cleveland Ave,4526,45.502960,-122.428030,105,41,8199.0,IN_TRANSIT_TO
9,12782283,0,2023-10-07 07:00:23,FX2 To Gresham,4522,45.527534,-122.675560,178,2,9301.0,IN_TRANSIT_TO
293,12782284,0,2023-10-07 07:09:14,FX2 To Gresham,3709,45.527695,-122.675570,178,1,9302.0,STOPPED_AT
...,...,...,...,...,...,...,...,...,...,...,...
55314,12782099,0,2023-10-08 05:30:38,FX2 To Gresham,4517,45.527695,-122.675570,178,1,9302.0,STOPPED_AT
55853,12782100,0,2023-10-08 05:46:43,FX2 To Gresham,4511,45.527695,-122.675570,178,1,9302.0,STOPPED_AT
54476,12782101,0,2023-10-08 06:15:33,FX2 To Gresham,4523,45.527695,-122.675570,178,1,9302.0,STOPPED_AT
54990,12782102,0,2023-10-08 06:34:35,FX2 To Gresham,4526,45.527695,-122.675570,178,1,9302.0,STOPPED_AT


In [265]:
df2.groupby("tripId").head(1)["vehicleStopSequence"].unique()

array([ 1,  2, 31, 10])